In [1]:
import os
import pandas as pd
data_dirpath = '/usr2/mamille2/fanfiction-project/'

# Evaluate/visualize document vectors from relationship tag

In [3]:
from gensim.models.doc2vec import Doc2Vec

model_dirpath = '/usr2/mamille2/fanfiction-project/models/'
fandom = 'detroit'
model_fpath = os.path.join(model_dirpath, fandom, 'PV-DM_d100w10n5mc2t20alpha0.05.model')

model = Doc2Vec.load(model_fpath)

## Word similarity evaluation

In [7]:
import random
from IPython.display import HTML

models = [model]

# pick a random word with a suitable number of occurences
while True:
    word = random.choice(models[0].wv.index2word)
    if models[0].wv.vocab[word].count > 50:
        break
        
# or uncomment below line, to just pick a word from the relevant domain:
# word = 'difficult'
similars_per_model = [str(model.wv.most_similar(word, topn=20)).replace('), ','),<br>\n') for model in models]
similar_table = ("<table><tr><th>" +
    "</th><th>".join([str(model) for model in models]) + 
    "</th></tr><tr><td>" +
    "</td><td>".join(similars_per_model) +
    "</td></tr></table>")
print("most similar words for '%s' (%d occurences)" % (word, models[0].wv.vocab[word].count))
HTML(similar_table)

most similar words for 'disguise' (304 occurences)


"Doc2Vec(""PV-DM_d100w10n5mc2t20alpha0.05"",dm/m,d100,n5,w10,mc2,t20)"
"[('mask', 0.673973798751831), ('hide', 0.654331624507904), ('rein', 0.6451747417449951), ('blend', 0.641869068145752), ('conceal', 0.6328172087669373), ('pose', 0.6302469968795776), ('uniform', 0.6174399852752686), ('suppress', 0.6090782284736633), ('sling', 0.608373761177063), ('place', 0.6041964888572693), ('dress', 0.60212641954422), ('costume', 0.5985825061798096), ('aide', 0.5977627635002136), ('suit', 0.5962046384811401), ('redeem', 0.5956819653511047), ('stifle', 0.5954487323760986), ('bind', 0.5921100974082947), ('tux', 0.5887809991836548), ('lockpick', 0.588256299495697), ('barcode', 0.5873401165008545)]"


## t-SNE

In [29]:
# Access word vectors, document vectors
# model['according']
model.docvecs['M/M']

array([-0.64145881, -0.44714788, -2.94131231,  1.95569456,  2.67215872,
       -1.21829247,  0.78172266,  1.58930683, -1.77131486,  0.48926157,
        1.09237397,  1.51042414, -1.2464937 ,  3.32916737,  0.86248803,
        0.34498975,  2.81938553,  1.60361052, -1.2919805 , -3.10155463,
       -0.97402817, -0.97601032, -2.3341856 , -2.05972171, -0.64949602,
        1.86308849,  0.80288702, -1.29127038, -0.55448472,  3.05143166,
       -2.26299334, -1.52647924,  1.74536729, -3.89703321, -1.45736158,
       -1.25314188, -0.34376073, -0.61410433, -0.25679395,  1.98801124,
        2.20992088, -1.44080317,  1.31203914, -0.39800346, -4.18623447,
        0.0956163 , -1.64500165, -0.04960196,  1.34296191, -0.72010326,
       -2.67911434, -1.00623536, -3.41361308, -1.03989995,  0.96937668,
       -0.63237178,  0.64114743, -0.66396761, -0.29518881, -2.77148938,
       -0.59896207, -1.04591382, -1.6616143 , -1.45333636,  0.3121596 ,
       -2.02744794, -0.57003492,  3.06457043, -0.10252312,  3.91

In [34]:
model.docvecs.doctags

{'F/F': Doctag(offset=3, word_count=308645, doc_count=8481),
 'F/M': Doctag(offset=2, word_count=1060994, doc_count=28695),
 'Gen': Doctag(offset=4, word_count=438631, doc_count=13483),
 'M/M': Doctag(offset=0, word_count=1195274, doc_count=34843),
 'Multi': Doctag(offset=1, word_count=335628, doc_count=10438),
 'Other': Doctag(offset=5, word_count=11834, doc_count=435)}

In [35]:
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt

def tsne_plot(model, random_subset=False):
    """Creates and TSNE model and plots it.
    Credit https://www.kaggle.com/jeffd23/visualizing-word-vectors-with-t-sne """
    
    labels = []
    tokens = []
    
#     if random_subset:
#         vocab = np.random.choice(list(model.wv.vocab.keys()), random_subset)
        
#     else:
#         vocab = model.wv.vocab
    vocab = model.wv.vocab
    
    for i, word in enumerate(vocab):
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    if random_subset:
        values_sample = np.random.choice(new_values, random_subset)
    else:
        values_sample = new_values
    for value in values_sample:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [12]:
len(model.wv.vocab)

22686

In [21]:
list(model.wv.vocab.keys())[400]

'gasping'

In [36]:
tsne_plot(model, random_subset=500)

KeyboardInterrupt: 

## Nearest neighbors for document vectors

In [8]:
model.docvecs.most_similar('F/M')

[('Other', 0.6788891553878784),
 ('F/F', 0.4212363064289093),
 ('Multi', 0.40247777104377747),
 ('', 0.25205597281455994),
 ('Gen', -0.021115733310580254),
 ('M/M', -0.4209712743759155)]

In [11]:
from pprint import pprint

freq_threshold = 100
n_top = 30

fandom = 'detroit'
outpath = os.path.join(data_dirpath, 'output', fandom, 'relationship_tags_nearest_neighbors.txt')

# cat = 'M/M'
# cat = 'F/F'
# cat = 'F/M'
# cat = 'Gen'
# cat = 'Multi'
# cat = 'Other'
with open(outpath, 'w') as f:
    for cat in sorted(model.docvecs.doctags):
        print(cat)
        f.write(f'{cat}\n')
        similar = model.wv.most_similar([model.docvecs[cat]], topn=300)
        similar_freq = [w for w in similar if model.wv.vocab[w[0]].count > freq_threshold][:n_top]
        pprint(similar_freq)
        for wd, sim in similar_freq:
            f.write(f'{wd}\t{sim}\n')
        print()
        f.write('\n')


[('safety', 0.5253168344497681),
 ('backyard', 0.5062938928604126),
 ('extent', 0.48983919620513916),
 ('basement', 0.48702695965766907),
 ('relation', 0.48562371730804443),
 ('contrast', 0.48470503091812134),
 ('mansion', 0.48414695262908936),
 ('house', 0.48316729068756104),
 ('mentor', 0.48109591007232666),
 ('fight', 0.4792930483818054),
 ('agreed', 0.4777863323688507),
 ('compound', 0.4748291075229645),
 ('progression', 0.47415244579315186),
 ('boat', 0.47396335005760193),
 ('monster', 0.47292253375053406),
 ('influence', 0.46788886189460754),
 ('dream', 0.46773114800453186),
 ('consequence', 0.4653666615486145),
 ('burden', 0.4634580910205841),
 ('rush', 0.46338772773742676),
 ('bounds', 0.463182270526886),
 ('creation', 0.462914377450943),
 ('shed', 0.4625244438648224),
 ('photograph', 0.4620272219181061),
 ('grave', 0.4619958698749542),
 ('workshop', 0.4617365300655365),
 ('bed', 0.46168577671051025),
 ('breed', 0.4606289863586426),
 ('survive', 0.46012720465660095),
 ('lied',

## See words in context

In [2]:
# Load data
fandom = 'detroit'
data_path = f'/usr2/mamille2/fanfiction-project/data/ao3/{fandom}/{fandom}_discoursedb_data.pkl'
data = pd.read_pickle(data_path)
print(len(data))
print(data.columns)

1356124
Index(['fic_id', 'chapter_id', 'para_id', 'text', 'preprocessed_text', 'title',
       'author', 'author_key', 'rating', 'category', 'fandom', 'relationship',
       'character', 'additional tags', 'language', 'published', 'status',
       'status date', 'words', 'comments', 'kudos', 'bookmarks', 'hits',
       'chapter_count'],
      dtype='object')


In [3]:
# Distribution of relationship types
from collections import Counter

value_counts = Counter([x for l in data['category'] for x in l])
value_counts.most_common()

[('M/M', 761964),
 ('F/M', 337608),
 ('Gen', 337401),
 ('Multi', 102706),
 ('F/F', 77378),
 ('Other', 74203),
 ('', 72851)]

In [4]:
pd.set_option('display.max_colwidth', -1)
import re

In [20]:
word = 'pillar'
regex = r'\b{}\b'.format(word)
data.loc[data['preprocessed_text'].map(lambda x: re.search(regex, x) is not None), ['preprocessed_text', 'category', 'relationship']].sample(50)

,preprocessed_text,category,relationship
1025075,"„ uf , okay , it feels like such a long time ago . well , my story is a simple one , i guess . we worked on this building , but it was a risky construction , based on the terrain we build on . one day , one of my co - workers get stucked under this pillar , his legs were stucked and the humans did not reacted to that in any way , but something in me just ... snapped . he and i , we talked sometimes and we mostly worked together . so , i freeded him on my own behalf , and after realizing what happened , before they could do anything to me , i run . and found this horrible man named zlatko . and the rest is a history . “ luther paused for a moment , josh nodded in understanding .","[F/M, M/M]","[""Kara/Luther"", ""Josh/Simon (Detroit: Become Human)"", ""Kara & Luther & Alice Williams (Detroit: Become Human)"", ""Josh & Luther""]"
386425,""" that 's up to pillar now , is n't it ? """,[Gen],[]
102829,"peering closer , connor looked the man up and down- his eyes glossed over . through a scan connor discovered he had received two bullets to the chest ( one perforating the vena cava and the other hitting him square in the lung- causing it to collapse ) and blunt force trauma from hitting his head against the pillar as he fell .",[M/M],"[""Hank Anderson/Connor"", ""Hank Anderson & Connor"", ""Hankcon""]"
686638,pillar collapsed to the ground and blood pooled beneath his head . hank let out the breath he had been holding and stood up from the ground as he stared at the body in the distance .,[Gen],[]
88032,"she twirled her pen almost violently in the elevator . the reality of things was starting to get to her . she was going to see him- she was going to see hank again , for the first time in almost three months . he thought she was dead . how would he react ? disbelief , to start , of course , but … would he yell ? panic ? go straight to denial ? she could n’t just not see him , but what if he did n’t want to see her ? what if- what if he had a new girlfriend ? that put a heavy weight in her gut , and if she could have lost anything about being human it would be that awful feeling of discontent settling low in her belly but it was apparently here to stay . she could n’t be mad , if he ’d moved on- he ’d been told she was dead , she could n’t expect him to have held on to a dead woman ’s memory , but- but what would she do with herself ? he ’d been such a pillar of her life , someone she relied on and trusted and loved- if he did n’t want her anymore-",[F/M],"[""Hank Anderson/Original Female Character(s)"", ""Connor & Original Female Character""]"
1401082,"when the machines had set her down , the android automatically turned on her synthetic skin and wandered around to look for a garment to cover her body . she soon found that the white room she was in contained only the bare necessities . aside from a few chairs , a well - worn painting , a glowing blue pillar , and two androids connected to big machines , she saw no signs of human life .",[F/M],"[""Connor/Kara (Detroit: Become Human)"", ""Kara|AX400/Connor|RK800"", ""Connara""]"
143192,"connor took a moment to appreciate gavin , he wanted to appreciate gavin one last time . gavin was honest , gavin was trying , and connor could n't be more proud . gavin was going to be okay , connor would make sure of that , and he knew that hank would make sure of it too long after connor was gone . at the same time , connor could n't help but feel sickeningly guilty . he was ripping away the one pillar of support that gavin had . he was ripping away another son from hank .",[M/M],"[""RK900 & Gavin Reed (Non-consensual)""]"
386496,""" this is it , connor . "" hank stated as he held his aim steady , his blue eyes focused on the cold hollow eyes on pillar in the distance .",[Gen],[]
686686,""" pillar shot you , too . this time it was because of me . """,[Gen],[]
386350,"connor 's l.e.d . blinked from the blue to yellow and then back to blue as he finish

# Prepare AO3 data in format for paragraph vector training

Preprocess content (tokenized, lowercased, like aclImdb/alldata-id.txt), extract relationship type metadata as tags

## Extract metadata

In [20]:
import pandas as pd

# Load in data from DiscourseDB
# data = pd.read_csv('/usr2/mamille2/fanfiction-project/data/ao3/friends/friends_discoursedb_data.csv')
# data = pd.read_pickle('/usr2/mamille2/fanfiction-project/data/ao3/friends/friends_discoursedb_data.pkl')
# metadata = pd.read_csv('/usr2/scratch/fanfic/ao3_academia_text/stories.csv')
metadata = pd.read_csv('/usr2/scratch/fanfic/ao3_detroit_text/stories.csv')
print(metadata.columns)
print(len(metadata))

Index(['fic_id', 'title', 'author', 'author_key', 'rating', 'category',
       'fandom', 'relationship', 'character', 'additional tags', 'language',
       'published', 'status', 'status date', 'words', 'comments', 'kudos',
       'bookmarks', 'hits', 'chapter_count'],
      dtype='object')
7108


In [21]:
# Convert string-lists to lists
metadata['category'] = metadata['category'].str.replace(r'[\[\]"]', '').str.split(',\s+')
metadata['category']

from collections import Counter

# Distribution of relationship types
value_counts = Counter([x for l in metadata['category'] for x in l])
value_counts.most_common()

[('M/M', 3955),
 ('Gen', 1757),
 ('F/M', 1240),
 ('', 534),
 ('Other', 398),
 ('Multi', 315),
 ('F/F', 210)]

## Merge in text data

In [22]:
from tqdm import tqdm_notebook as tqdm
import os
import numpy as np

# story_dirpath = '/usr2/scratch/fanfic/ao3_academia_text/stories'
story_dirpath = '/usr2/scratch/fanfic/ao3_detroit_text/stories'

texts = []

# Load header
hdr = pd.read_csv(os.path.join(story_dirpath, os.listdir(story_dirpath)[0])).columns

# Load data
for fname in tqdm(os.listdir(story_dirpath)):
    story_df = pd.read_csv(os.path.join(story_dirpath, fname))
    texts.append(story_df.values)

In [23]:
import numpy as np

# Merge
data = pd.DataFrame(np.vstack(texts), columns=hdr)
print(len(data))
data.columns

1469142


Index(['fic_id', 'chapter_id', 'para_id', 'text'], dtype='object')

In [24]:
import spacy
nlp = spacy.load('en')

def preprocess_text(text):
    if isinstance(text, float):
        return ''
    else:
        return ' '.join([tok.text for tok in nlp.tokenizer(text.lower().replace('\n', ' '))]).strip()

In [25]:
from tqdm import tqdm_notebook as tqdm
data['preprocessed_text'] = list(map(preprocess_text, tqdm(data['text'])))
# data['preprocessed_text']

data['preprocessed_text'].sample(3)

448200     “ you said last week that i was “ sucking the ...
1280187    рид , кажется , куда-то выпадает , замерев над...
984482     the next morning , connor once more uses the s...
Name: preprocessed_text, dtype: object

In [26]:
print(data.columns)
print(len(data))
print(metadata.columns)

Index(['fic_id', 'chapter_id', 'para_id', 'text', 'preprocessed_text'], dtype='object')
1469142
Index(['fic_id', 'title', 'author', 'author_key', 'rating', 'category',
       'fandom', 'relationship', 'character', 'additional tags', 'language',
       'published', 'status', 'status date', 'words', 'comments', 'kudos',
       'bookmarks', 'hits', 'chapter_count'],
      dtype='object')


In [27]:
# Merge metadata and text data
merged = pd.merge(data, metadata, on=['fic_id'], how='left')
print(len(merged))
print(merged.columns)

1469142
Index(['fic_id', 'chapter_id', 'para_id', 'text', 'preprocessed_text', 'title',
       'author', 'author_key', 'rating', 'category', 'fandom', 'relationship',
       'character', 'additional tags', 'language', 'published', 'status',
       'status date', 'words', 'comments', 'kudos', 'bookmarks', 'hits',
       'chapter_count'],
      dtype='object')


In [29]:
# Filter to English
merged = merged[merged['language']=='English']
print(len(merged))

1356124


## Extract annotations (not necessary for data directly in /usr2/scratch/fanfic)

In [87]:
pd.set_option('display.max_colwidth', -1)

data['annotations']

In [16]:
import re
# from IPython.core.debugger import set_trace

# Extract relationship type column
def extract_annotation_category(category, annotations):
    annotations_dict = {el[0]: ''.join(el[1:]) for el in [b.split(' (') for b in annotations.split(', ')]}
    try:
        vals = re.findall(r'null=(.*?)[;\)]', annotations_dict[category])
    except:
        set_trace()
    return vals

In [17]:
data['relationship_type'] = [extract_annotation_category('category', x) for x in tqdm(data['annotations'].tolist())]
data['relationship_type']

0        [F/M, M/M]
1        [F/M, M/M]
2        [F/M, M/M]
3        [M/M, F/M]
4        [M/M, F/M]
5        [F/M, M/M]
6        [M/M, F/M]
7        [M/M, F/M]
8        [M/M, F/M]
9        [M/M, F/M]
10       [M/M, F/M]
11       [M/M, F/M]
12       [M/M, F/M]
13       [F/M, M/M]
14       [M/M, F/M]
15       [M/M, F/M]
16       [M/M, F/M]
17       [M/M, F/M]
18       [M/M, F/M]
19       [M/M, F/M]
20       [M/M, F/M]
21       [F/M, M/M]
22       [M/M, F/M]
23       [M/M, F/M]
24       [M/M, F/M]
25       [M/M, F/M]
26       [M/M, F/M]
27       [M/M, F/M]
28       [M/M, F/M]
29       [F/M, M/M]
            ...    
74169         [F/M]
74170         [F/M]
74171         [F/M]
74172         [F/M]
74173         [F/M]
74174         [F/M]
74175         [F/M]
74176         [F/M]
74177         [F/M]
74178         [F/M]
74179         [F/M]
74180         [F/M]
74181         [F/M]
74182         [F/M]
74183         [F/M]
74184         [F/M]
74185         [F/M]
74186         [F/M]
74187         [F/M]


## Save out annotations, preprocessed content

In [30]:
# outpath = '/usr2/mamille2/fanfiction-project/data/ao3/friends/friends_discoursedb_data.pkl'
# outpath = '/usr2/mamille2/fanfiction-project/data/ao3/academia/academia_discoursedb_data.pkl'
outpath = '/usr2/mamille2/fanfiction-project/data/ao3/detroit/detroit_discoursedb_data.pkl'
merged.to_pickle(outpath)

## Save out documents to their own lines (optional)

In [47]:
# Save paragraphs to their own lines in file
with open('/usr2/mamille2/fanfiction-project/data/ao3/friends/friends_paragraphs.txt', 'wb') as f:
    for p in data['preprocessed_content'].tolist():
        f.write(f'{p}\n'.encode('utf8'))
#         f.write('aééé'.encode('utf8').decode('utf8'))
#         f.write('aééé'.encode('utf8'))

# Gensim Doc2Vec with AO3 data (documents tagged with relationship type)
From https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-IMDB.ipynb

In [20]:
%%time

import gensim
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple
from smart_open import smart_open

fpath = '/usr2/mamille2/fanfiction-project/data/ao3/friends/friends_discoursedb_data.pkl'
test_fraction = 0.1

data = pd.read_pickle(fpath)
total_docs = len(data)

alldocs = []
for line, tags in zip(data['preprocessed_content'], data['relationship_type']):
    tokens = gensim.utils.to_unicode(line).split()
    alldocs.append(TaggedDocument(words, tags))

# train_docs = [doc for doc in alldocs if doc.split == 'train']
# test_docs = [doc for doc in alldocs if doc.split == 'test']

# print('%d docs: %d train, %d test' % (len(alldocs), len(train_docs), len(test_docs)))

from random import shuffle
doc_list = alldocs[:]  
shuffle(doc_list)

print(len(doc_list))

74199
CPU times: user 776 ms, sys: 148 ms, total: 924 ms
Wall time: 927 ms


In [21]:
%%time

# Build models

from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

# cores = multiprocessing.cpu_count()
cores = 20
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

simple_models = [
    # PV-DBOW plain
    Doc2Vec(dm=0, vector_size=100, negative=5, hs=0, min_count=2, sample=0, 
            epochs=20, workers=cores),
    # PV-DM w/ default averaging; a higher starting alpha may improve CBOW/PV-DM modes
    Doc2Vec(dm=1, vector_size=100, window=10, negative=5, hs=0, min_count=2, sample=0, 
            epochs=20, workers=cores, alpha=0.05, comment='alpha=0.05'),
    # PV-DM w/ concatenation - big, slow, experimental mode
    # window=5 (both sides) approximates paper's apparent 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, vector_size=100, window=5, negative=5, hs=0, min_count=2, sample=0, 
            epochs=20, workers=cores),
]

for model in simple_models:
    model.build_vocab(alldocs)
    print("%s vocabulary scanned & state initialized" % model)

models_by_name = OrderedDict((str(model), model) for model in simple_models)

from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[1]])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[2]])

Doc2Vec(dbow,d100,n5,mc2,t20) vocabulary scanned & state initialized
Doc2Vec("alpha=0.05",dm/m,d100,n5,w10,mc2,t20) vocabulary scanned & state initialized
Doc2Vec(dm/c,d100,n5,w5,mc2,t20) vocabulary scanned & state initialized
CPU times: user 6min 14s, sys: 1.27 s, total: 6min 16s
Wall time: 6min 16s


## Train models

In [ ]:
for model in simple_models: 
    print("Training %s" % model)
    %time model.train(doc_list, total_examples=len(doc_list), epochs=model.epochs) # Adjust epochs in model 
    print()

Training Doc2Vec(dbow,d100,n5,mc2,t20)


KeyboardInterrupt: 


Training Doc2Vec("alpha=0.05",dm/m,d100,n5,w10,mc2,t20)


## Qualitative word vector evaluation

In [12]:
import random
from IPython.display import HTML

# pick a random word with a suitable number of occurences
while True:
    word = random.choice(simple_models[0].wv.index2word)
    if simple_models[0].wv.vocab[word].count > 10:
        break
        
# or uncomment below line, to just pick a word from the relevant domain:
# word = 'difficult'
similars_per_model = [str(model.wv.most_similar(word, topn=20)).replace('), ','),<br>\n') for model in simple_models]
similar_table = ("<table><tr><th>" +
    "</th><th>".join([str(model) for model in simple_models]) + 
    "</th></tr><tr><td>" +
    "</td><td>".join(similars_per_model) +
    "</td></tr></table>")
print("most similar words for '%s' (%d occurences)" % (word, simple_models[0].wv.vocab[word].count))
HTML(similar_table)

most similar words for 'smashing' (20 occurences)


"Doc2Vec(dbow,d100,n5,mc2,t20)","Doc2Vec(""alpha=0.05"",dm/m,d100,n5,w10,mc2,t20)","Doc2Vec(dm/c,d100,n5,w5,mc2,t20)"
"[('mutations', 0.39711183309555054), ('stealthily', 0.39297765493392944), ('fondest', 0.38682976365089417), ('surfer', 0.3744651675224304), ('discouraging', 0.35691094398498535), ('dotted', 0.35466301441192627), ('uahan', 0.3428993225097656), ('rty', 0.342846542596817), ('thirds', 0.34089499711990356), ('shielding', 0.3404473662376404), ('authentic', 0.3356667757034302), ('lent', 0.3335157632827759), ('worshipped', 0.3328370749950409), ('plunged', 0.33239153027534485), ('engaging', 0.33150768280029297), ('slinks', 0.3224559724330902), ('inform', 0.32213205099105835), ('preps', 0.32165539264678955), ('acquaintances', 0.3209088146686554), ('claudia', 0.318649023771286)]","[('mutations', 0.39711180329322815), ('stealthily', 0.39297762513160706), ('fondest', 0.38682979345321655), ('surfer', 0.37446513772010803), ('discouraging', 0.3569110035896301), ('dotted', 0.35466307401657104), ('uahan', 0.3428993225097656), ('rty', 0.34284651279449463), ('thirds', 0.34089499711990356), ('shielding', 0.34044742584228516), ('lent', 0.3335157632827759), ('worshipped', 0.3328370749950409), ('plunged', 0.33239153027534485), ('engaging', 0.33150771260261536), ('slinks', 0.3224559724330902), ('inform', 0.32213202118873596), ('acquaintances', 0.3209088444709778), ('coffee', 0.31975361704826355), ('claudia', 0.3186489939689636), ('advancing', 0.3178688883781433)]","[('hugh', 0.40148839354515076), ('mutations', 0.39711180329322815), ('stealthily', 0.39297765493392944), ('fondest', 0.38682976365089417), ('surfer', 0.3744651675224304), ('orders', 0.35907721519470215), ('discouraging', 0.35691094398498535), ('dotted', 0.35466301441192627), ('uahan', 0.3428993225097656), ('rty', 0.342846542596817), ('thirds', 0.34089499711990356), ('shielding', 0.3404473662376404), ('uninhibited', 0.33743056654930115), ('lent', 0.3335157632827759), ('natural', 0.33288058638572693), ('worshipped', 0.3328370749950409), ('plunged', 0.33239153027534485), ('engaging', 0.33150768280029297), ('human', 0.329820841550827), ('slinks', 0.3224559724330902)]"


# Test Gensim Doc2Vec with AO3 data (document = paragraph)
From https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-IMDB.ipynb

In [49]:
%%time

import gensim
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple

fpath = '/usr2/mamille2/fanfiction-project/data/ao3/friends/friends_paragraphs.txt'
test_fraction = 0.1
total_docs = 74199

# this data object class suffices as a `TaggedDocument` (with `words` and `tags`) 
# plus adds other state helpful for our later evaluation/reporting
TaggedDocument = namedtuple('TaggedDocument', 'words tags split')

alldocs = []
with smart_open(fpath, 'rb', encoding='utf-8') as alldata:
    for line_no, line in enumerate(alldata):
        tokens = gensim.utils.to_unicode(line).split()
        words = tokens[1:]
        tags = [line_no] # tags are labels; 'tags = [tokens[0]]' would also work at extra memory cost
        split = ['train', 'test'][line_no // int(total_docs * (1-test_fraction))]
        alldocs.append(TaggedDocument(words, tags, split))

train_docs = [doc for doc in alldocs if doc.split == 'train']
test_docs = [doc for doc in alldocs if doc.split == 'test']

print('%d docs: %d train, %d test' % (len(alldocs), len(train_docs), len(test_docs)))

from random import shuffle
doc_list = alldocs[:]  
shuffle(doc_list)

In [50]:
%%time

# Build models

from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

# cores = multiprocessing.cpu_count()
cores = 20
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

simple_models = [
    # PV-DBOW plain
    Doc2Vec(dm=0, vector_size=100, negative=5, hs=0, min_count=2, sample=0, 
            epochs=20, workers=cores),
    # PV-DM w/ default averaging; a higher starting alpha may improve CBOW/PV-DM modes
    Doc2Vec(dm=1, vector_size=100, window=10, negative=5, hs=0, min_count=2, sample=0, 
            epochs=20, workers=cores, alpha=0.05, comment='alpha=0.05'),
    # PV-DM w/ concatenation - big, slow, experimental mode
    # window=5 (both sides) approximates paper's apparent 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, vector_size=100, window=5, negative=5, hs=0, min_count=2, sample=0, 
            epochs=20, workers=cores),
]

for model in simple_models:
    model.build_vocab(alldocs)
    print("%s vocabulary scanned & state initialized" % model)

models_by_name = OrderedDict((str(model), model) for model in simple_models)

from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[1]])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[2]])

Doc2Vec(dbow,d100,n5,mc2,t20) vocabulary scanned & state initialized
Doc2Vec("alpha=0.05",dm/m,d100,n5,w10,mc2,t20) vocabulary scanned & state initialized
Doc2Vec(dm/c,d100,n5,w5,mc2,t20) vocabulary scanned & state initialized
CPU times: user 9.44 s, sys: 664 ms, total: 10.1 s
Wall time: 10.1 s


## Train models

In [51]:
for model in simple_models: 
    print("Training %s" % model)
    %time model.train(doc_list, total_examples=len(doc_list), epochs=model.epochs)
    print()

Training Doc2Vec(dbow,d100,n5,mc2,t20)
CPU times: user 3min 16s, sys: 18.4 s, total: 3min 34s
Wall time: 1min 51s

Training Doc2Vec("alpha=0.05",dm/m,d100,n5,w10,mc2,t20)
CPU times: user 5min 17s, sys: 55.2 s, total: 6min 12s
Wall time: 2min 47s

Training Doc2Vec(dm/c,d100,n5,w5,mc2,t20)
CPU times: user 8min 27s, sys: 25.1 s, total: 8min 52s
Wall time: 2min 13s



In [56]:
import random
from IPython.display import HTML

# pick a random word with a suitable number of occurences
while True:
    word = random.choice(simple_models[0].wv.index2word)
    if simple_models[0].wv.vocab[word].count > 10:
        break
        
# or uncomment below line, to just pick a word from the relevant domain:
# word = 'spoilt'
similars_per_model = [str(model.wv.most_similar(word, topn=20)).replace('), ','),<br>\n') for model in simple_models]
similar_table = ("<table><tr><th>" +
    "</th><th>".join([str(model) for model in simple_models]) + 
    "</th></tr><tr><td>" +
    "</td><td>".join(similars_per_model) +
    "</td></tr></table>")
print("most similar words for '%s' (%d occurences)" % (word, simple_models[0].wv.vocab[word].count))
HTML(similar_table)

most similar words for 'murmur' (13 occurences)


"Doc2Vec(dbow,d100,n5,mc2,t20)","Doc2Vec(""alpha=0.05"",dm/m,d100,n5,w10,mc2,t20)","Doc2Vec(dm/c,d100,n5,w5,mc2,t20)"
"[('unsuspecting', 0.39443328976631165), ('cellphone', 0.3796854019165039), ('iffy', 0.372952401638031), ('excess', 0.3608635663986206), ('redford', 0.3578962981700897), ('hints', 0.3546637296676636), ('activated', 0.34775805473327637), ('happenings', 0.3403438925743103), ('scraping', 0.33970412611961365), ('with', 0.3392728269100189), ('dialled', 0.33752650022506714), ('flavor', 0.33679360151290894), ('è', 0.3287610709667206), ('on-', 0.32636886835098267), ('kämpfer', 0.32536137104034424), ('cowardice', 0.3245278596878052), ('naturedly', 0.3223896622657776), ('employers', 0.32194066047668457), ('absolution', 0.31825727224349976), ('valasus', 0.3180891275405884)]","[('whisper', 0.6813105344772339), ('growl', 0.5926134586334229), ('coward', 0.5922876000404358), ('moment', 0.573410153388977), ('burglar', 0.5642147064208984), ('mover', 0.5640881657600403), ('teenager', 0.5636329054832458), ('minute', 0.5632354021072388), ('halt', 0.5546646118164062), ('sob', 0.5538667440414429), ('skleničku', 0.5502578616142273), ('continent', 0.5448827743530273), ('huff', 0.5434749722480774), ('mockery', 0.5430886745452881), ('word', 0.5400181412696838), ('sniffle', 0.5378166437149048), ('click', 0.5377556681632996), ('volver', 0.5347557663917542), ('participant', 0.5343178510665894), ('bit', 0.5334388613700867)]","[('enrol', 0.6278350949287415), ('twig', 0.6177855730056763), ('snort', 0.6093369126319885), ('halt', 0.6067569255828857), ('fight.’', 0.5970606803894043), ('revolt', 0.591293215751648), ('yawn', 0.5861109495162964), ('sob', 0.5846139192581177), ('wedge', 0.5762349963188171), ('swerve', 0.5714218616485596), ('nestle', 0.5631072521209717), ('caress', 0.5629807710647583), ('miscommunication', 0.5629664659500122), ('sooth', 0.5585294365882874), ('rook', 0.5577542781829834), ('puddle', 0.5549665093421936), ('connoisseur', 0.5526505708694458), ('tweak', 0.5496055483818054), ('yuki', 0.547900378704071), ('swat', 0.546383261680603)]"


## Do close documents seem more related than distant ones?

In [60]:
import random
import numpy as np

doc_id = np.random.randint(simple_models[0].docvecs.count)  # pick random doc, re-run cell for more examples
model = random.choice(simple_models)  # and a random model
sims = model.docvecs.most_similar(doc_id, topn=model.docvecs.count)  # get *all* similar documents
print(u'TARGET (%d): «%s»\n' % (doc_id, ' '.join(alldocs[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(alldocs[sims[index][0]].words)))

TARGET (37520): «alright ... " the blond stares at nico before suddenly inhaling sharply . " well , there you go ... "»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec("alpha=0.05",dm/m,d100,n5,w10,mc2,t20):

MOST (74142, 0.7427983283996582): «nodded again . " okay . "»

MEDIAN (32786, 0.407391756772995): «closes the door behind her , and joey shakes his head . thank god rachel did n’t notice anything different about him .»

LEAST (40699, -0.3952307403087616): «_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _»



# Test Gensim Doc2Vec with provided data
From https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-IMDB.ipynb

In [1]:
%%time 

import locale
import glob
import os.path
import requests
import tarfile
import sys
import codecs
from smart_open import smart_open
import re

dirname = 'aclImdb'
filename = 'aclImdb_v1.tar.gz'
locale.setlocale(locale.LC_ALL, 'C')
all_lines = []

if sys.version > '3':
    control_chars = [chr(0x85)]
else:
    control_chars = [unichr(0x85)]

# Convert text to lower-case and strip punctuation/symbols from words
def normalize_text(text):
    norm_text = text.lower()
    # Replace breaks with spaces
    norm_text = norm_text.replace('<br />', ' ')
    # Pad punctuation with spaces on both sides
    norm_text = re.sub(r"([\.\",\(\)!\?;:])", " \\1 ", norm_text)
    return norm_text

if not os.path.isfile('aclImdb/alldata-id.txt'):
    if not os.path.isdir(dirname):
        if not os.path.isfile(filename):
            # Download IMDB archive
            print("Downloading IMDB archive...")
            url = u'http://ai.stanford.edu/~amaas/data/sentiment/' + filename
            r = requests.get(url)
            with smart_open(filename, 'wb') as f:
                f.write(r.content)
        # if error here, try `tar xfz aclImdb_v1.tar.gz` outside notebook, then re-run this cell
        tar = tarfile.open(filename, mode='r')
        tar.extractall()
        tar.close()
    else:
        print("IMDB archive directory already available without download.")

    # Collect & normalize test/train data
    print("Cleaning up dataset...")
    folders = ['train/pos', 'train/neg', 'test/pos', 'test/neg', 'train/unsup']
    for fol in folders:
        temp = u''
        newline = "\n".encode("utf-8")
        output = fol.replace('/', '-') + '.txt'
        # Is there a better pattern to use?
        txt_files = glob.glob(os.path.join(dirname, fol, '*.txt'))
        print(" %s: %i files" % (fol, len(txt_files)))
        with smart_open(os.path.join(dirname, output), "wb") as n:
            for i, txt in enumerate(txt_files):
                with smart_open(txt, "rb") as t:
                    one_text = t.read().decode("utf-8")
                    for c in control_chars:
                        one_text = one_text.replace(c, ' ')
                    one_text = normalize_text(one_text)
                    all_lines.append(one_text)
                    n.write(one_text.encode("utf-8"))
                    n.write(newline)

    # Save to disk for instant re-use on any future runs
    with smart_open(os.path.join(dirname, 'alldata-id.txt'), 'wb') as f:
        for idx, line in enumerate(all_lines):
            num_line = u"_*{0} {1}\n".format(idx, line)
            f.write(num_line.encode("utf-8"))

assert os.path.isfile("aclImdb/alldata-id.txt"), "alldata-id.txt unavailable"
print("Success, alldata-id.txt is available for next steps.")

Success, alldata-id.txt is available for next steps.
CPU times: user 240 ms, sys: 40 ms, total: 280 ms
Wall time: 281 ms


In [2]:
%%time

import gensim
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple

# this data object class suffices as a `TaggedDocument` (with `words` and `tags`) 
# plus adds other state helpful for our later evaluation/reporting
SentimentDocument = namedtuple('SentimentDocument', 'words tags split sentiment')

alldocs = []
with smart_open('aclImdb/alldata-id.txt', 'rb', encoding='utf-8') as alldata:
    for line_no, line in enumerate(alldata):
        tokens = gensim.utils.to_unicode(line).split()
        words = tokens[1:]
        tags = [line_no] # 'tags = [tokens[0]]' would also work at extra memory cost
        split = ['train', 'test', 'extra', 'extra'][line_no//25000]  # 25k train, 25k test, 25k extra
        sentiment = [1.0, 0.0, 1.0, 0.0, None, None, None, None][line_no//12500] # [12.5K pos, 12.5K neg]*2 then unknown
        alldocs.append(SentimentDocument(words, tags, split, sentiment))

train_docs = [doc for doc in alldocs if doc.split == 'train']
test_docs = [doc for doc in alldocs if doc.split == 'test']

print('%d docs: %d train-sentiment, %d test-sentiment' % (len(alldocs), len(train_docs), len(test_docs)))

100000 docs: 25000 train-sentiment, 25000 test-sentiment
CPU times: user 8.72 s, sys: 1.27 s, total: 9.99 s
Wall time: 10 s


In [3]:
from random import shuffle
doc_list = alldocs[:]  
shuffle(doc_list)

In [4]:
%%time
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

# cores = multiprocessing.cpu_count()
cores = 20
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

simple_models = [
    # PV-DBOW plain
    Doc2Vec(dm=0, vector_size=100, negative=5, hs=0, min_count=2, sample=0, 
            epochs=20, workers=cores),
    # PV-DM w/ default averaging; a higher starting alpha may improve CBOW/PV-DM modes
    Doc2Vec(dm=1, vector_size=100, window=10, negative=5, hs=0, min_count=2, sample=0, 
            epochs=20, workers=cores, alpha=0.05, comment='alpha=0.05'),
    # PV-DM w/ concatenation - big, slow, experimental mode
    # window=5 (both sides) approximates paper's apparent 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, vector_size=100, window=5, negative=5, hs=0, min_count=2, sample=0, 
            epochs=20, workers=cores),
]

for model in simple_models:
    model.build_vocab(alldocs)
    print("%s vocabulary scanned & state initialized" % model)

models_by_name = OrderedDict((str(model), model) for model in simple_models)

Doc2Vec(dbow,d100,n5,mc2,t20) vocabulary scanned & state initialized
Doc2Vec("alpha=0.05",dm/m,d100,n5,w10,mc2,t20) vocabulary scanned & state initialized
Doc2Vec(dm/c,d100,n5,w5,mc2,t20) vocabulary scanned & state initialized
CPU times: user 40.8 s, sys: 4.29 s, total: 45.1 s
Wall time: 45.1 s


In [5]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[1]])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[2]])

## Train models

In [6]:
for model in simple_models: 
    print("Training %s" % model)
    %time model.train(doc_list, total_examples=len(doc_list), epochs=model.epochs)
    print()

Training Doc2Vec(dbow,d100,n5,mc2,t20)
CPU times: user 25min 13s, sys: 58 s, total: 26min 11s
Wall time: 6min 20s
Training Doc2Vec("alpha=0.05",dm/m,d100,n5,w10,mc2,t20)
CPU times: user 53min 28s, sys: 2min 32s, total: 56min 1s
Wall time: 9min 54s
Training Doc2Vec(dm/c,d100,n5,w5,mc2,t20)
CPU times: user 2h 49s, sys: 2min 42s, total: 2h 3min 31s
Wall time: 9min 46s


In [7]:
import random
from IPython.display import HTML

# pick a random word with a suitable number of occurences
# while True:
#     word = random.choice(simple_models[0].wv.index2word)
#     if simple_models[0].wv.vocab[word].count > 10:
#         break
        
# or uncomment below line, to just pick a word from the relevant domain:
word = 'spoilt'
similars_per_model = [str(model.wv.most_similar(word, topn=20)).replace('), ','),<br>\n') for model in simple_models]
similar_table = ("<table><tr><th>" +
    "</th><th>".join([str(model) for model in simple_models]) + 
    "</th></tr><tr><td>" +
    "</td><td>".join(similars_per_model) +
    "</td></tr></table>")
print("most similar words for '%s' (%d occurences)" % (word, simple_models[0].wv.vocab[word].count))
HTML(similar_table)

most similar words for 'spoilt' (97 occurences)


"Doc2Vec(dbow,d100,n5,mc2,t20)","Doc2Vec(""alpha=0.05"",dm/m,d100,n5,w10,mc2,t20)","Doc2Vec(dm/c,d100,n5,w5,mc2,t20)"
"[('ballet', 0.4222918152809143), ('65-minute', 0.41812703013420105), ('nit-wit', 0.38984495401382446), ('action-comedy', 0.3879348337650299), ('pimped', 0.37372422218322754), ('matara', 0.3736003637313843), ('brownstone', 0.37183868885040283), (""dick's"", 0.36779266595840454), (""rodger's"", 0.367227703332901), ('verano', 0.3622084856033325), ('consummately', 0.3592488169670105), (""macready's"", 0.3578265309333801), ('*such*', 0.3577822148799896), ('krassin', 0.3576417863368988), ('risks', 0.35670018196105957), ('callings', 0.35649171471595764), ('aic', 0.35543423891067505), ('distracts', 0.3541957139968872), (""freaks'"", 0.35217738151550293), ('ronreaco', 0.35198500752449036)]","[('spoiled', 0.6510179042816162), ('undermined', 0.5426546335220337), ('dominated', 0.5321130752563477), ('ruined', 0.5287238955497742), ('over-shadowed', 0.5230126976966858), ('marred', 0.5212410688400269), ('replaced', 0.5141274333000183), ('followed', 0.4941771924495697), ('siring', 0.49131008982658386), ('rejected', 0.4876922070980072), ('dwarfed', 0.48443150520324707), ('unencumbered', 0.4830629825592041), ('snatched', 0.479458749294281), ('populated', 0.47769853472709656), ('dazzled', 0.4664037823677063), ('inhabited', 0.46631473302841187), ('entranced', 0.4659886360168457), ('surrounded', 0.46457332372665405), ('bolstered', 0.463314026594162), ('saved', 0.46262553334236145)]","[('spoiled', 0.6605638265609741), ('ruined', 0.5010989904403687), ('disturbed', 0.49336788058280945), ('plagued', 0.48810648918151855), ('wooed', 0.48625805974006653), ('dampened', 0.4841140806674957), ('lightened', 0.4838625490665436), ('shaken', 0.47890323400497437), ('troubled', 0.4785526990890503), ('torpedoed', 0.47636711597442627), ('racked', 0.4725381135940552), ('maltreated', 0.4714895784854889), ('wounded', 0.46761342883110046), ('lessened', 0.4648064076900482), ('drugged', 0.46392497420310974), ('stimulated', 0.46230459213256836), ('breakable', 0.46102404594421387), ('saved', 0.4606311619281769), ('cuckolded', 0.4579746723175049), ('willed', 0.455874502658844)]"
